In [4]:
from __future__ import print_function
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.preprocessing import MinMaxScaler
print(tf.__version__)
import sys
print(sys.version)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

%matplotlib inline


2.8.2
3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [5]:
data = pd.read_csv("/content/in_domain_train.csv")
data_dev = pd.read_csv("/content/in_domain_dev.csv")
data = data.values
data_dev = data_dev.values
x_train = data[:,1]
y_train = data[:,2]

x_dev = data_dev[:,1]
y_dev = data_dev[:,2]

y = np.concatenate((y_dev,y_train))
x = np.concatenate((x_dev,x_train))
import random
random.shuffle(y)
random.shuffle(x)  
y_t_0 = y[1500:]
y_d_0 = y[:1500]
x_t_0 = x[1500:]
x_d_0 = x[:1500]

In [6]:
f = open('entries.txt')
lines=f.readlines()
sentences = []
y = []
line = 0
cur = 0
for x in lines:
    cur = cur +1
    if x=='\n' : continue
    else: line=line+1
    if line>=60000: 
        print(x,cur)
        break
    sentences.append(x.split('\t')[4])
    if x.split('\t')[0] == '0':
        y.append(1)
    else:
        y.append(0)
f.close()

0	307365	http://lang-8.com/18840/journals/84655	21	They asked me to explain `` Ionic Bond `` in English .
 65781


In [7]:
print(len(y),len(sentences))

59999 59999


In [8]:
zeros = 0
ones = 0
for l in y:
    if l==0:
        zeros = zeros+1
    else:
        ones = ones+1
print(zeros,ones)

29389 30610


In [9]:
y_t_1 = y[5000:]
y_d_1 = y[:5000]
x_t_1 = sentences[5000:]
x_d_1 = sentences[:5000]

y_t = np.concatenate((y_t_1,y_t_0.tolist()))
y_d = np.concatenate((y_d_1,y_d_0.tolist()))
sentences_t = np.concatenate((x_t_1,x_t_0))
sentences_d = np.concatenate((x_d_1,x_d_0))

print(len(y_t),len(y_d),sentences_t[:3],sentences_d[:3])

62351 6500 ['I really miss them .\n' "I 'm not sure how I spend more 3 weeks here ."
 'Oh my god !\n'] ['Good luck on your new start !\n'
 'My teacher is going to move to change his job .\n'
 'He is a so nice guy and taught me English very kindly and was willing to accept my getting off the track .\n']


In [10]:
embedding_dim = 100
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences_t)

vocab_size = len(tokenizer.word_index)+1
sequences = tokenizer.texts_to_sequences(sentences_t)
padded = pad_sequences(sequences,maxlen=max_length)

testing_sequences = tokenizer.texts_to_sequences(sentences_d)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

y_train_np = np.array(y_t,np.float32)
y_dev_np = np.array(y_d,np.float32)
testing_padded_np = np.array(testing_padded,np.float32)
padded_np = np.array(padded,np.float32)

y_train_tf = tf.convert_to_tensor(y_train_np, np.float32)
y_dev_tf = tf.convert_to_tensor(y_dev_np, np.float32)
testing_padded_tf = tf.convert_to_tensor(testing_padded_np, np.float32)
padded_tf = tf.convert_to_tensor(padded_np, np.float32)

In [11]:
print(padded_np)

[[0.0000e+00 0.0000e+00 0.0000e+00 ... 9.1000e+01 6.9100e+02 7.0000e+01]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 2.3000e+02 6.3200e+02 1.3600e+02]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 4.2400e+02 7.0000e+00 8.4100e+02]
 ...
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 3.8682e+04 3.8683e+04 4.3190e+03]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 3.8687e+04 3.8688e+04 4.6840e+03]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 3.0100e+02 3.8689e+04 1.1218e+04]]


In [12]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.array(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 283597 word vectors.


In [13]:
count = 0
for word,i in tokenizer.word_index.items():
    cur = cur+1
print(cur)

104470


In [14]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [15]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False),
    tf.keras.layers.LSTM(64,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss = 'binary_crossentropy', optimizer=opt,metrics = ['accuracy'])
print(model.summary())
print(y_dev_tf.shape,y_train_tf.shape,padded_tf.shape,testing_padded_tf.shape)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 100)          3869000   
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
 dense_1 (Dense)             (None, 1)                 6         
                                                        

In [16]:
print(testing_padded_tf.shape,y_dev_tf.shape,padded_tf.shape,y_train_tf.shape)

(6500, 120) (6500,) (62351, 120) (62351,)


In [17]:
num_epochs = 5
model.fit(padded_tf, y_train_tf, epochs=num_epochs, validation_data=(testing_padded_tf, y_dev_tf))

Epoch 1/5
1949/1949 [==============================] - 369s 187ms/step - loss: 0.6890 - accuracy: 0.6038 - val_loss: 0.6360 - val_accuracy: 0.6248
Epoch 2/5
1949/1949 [==============================] - 363s 186ms/step - loss: 0.6391 - accuracy: 0.6232 - val_loss: 0.6238 - val_accuracy: 0.6357
Epoch 3/5
1949/1949 [==============================] - 365s 187ms/step - loss: 0.6310 - accuracy: 0.6279 - val_loss: 0.6207 - val_accuracy: 0.6435
Epoch 4/5
1949/1949 [==============================] - 360s 184ms/step - loss: 0.6267 - accuracy: 0.6348 - val_loss: 0.6252 - val_accuracy: 0.6588
Epoch 5/5
1949/1949 [==============================] - 362s 186ms/step - loss: 0.6222 - accuracy: 0.6409 - val_loss: 0.6162 - val_accuracy: 0.6540


In [19]:
custom = input("Sentence: ")
custom_test_1 = np.array([custom])
custom_1 = tokenizer.texts_to_sequences(custom_test_1)
flag = 0
for q in custom_1[0]:
    if q!=1: 
        flag = 1
        break
if flag == 0: 
    print("All words out of vocabulary!")
custom_2 = pad_sequences(custom_1,maxlen=max_length)
custom_np = np.array(custom_2,np.float32)
custom_tf = tf.convert_to_tensor(custom_np,np.float32)
prob = model.predict(custom_tf)
if prob<0.65:
    print("Grammatically \033[1m INCORRECT \033[0m with probability: ",1-prob)
else:
    print("Grammatically \033[1m CORRECT \033[0m with probability: ",prob)
print(prob)
print(custom_test_1)

Sentence: This is a wrong sentence.
Grammatically  CORRECT  with probability:  [[0.6559517]]
[[0.6559517]]
['This is a wrong sentence.']
